In [ ]:
from datasets import load_dataset
from datasets import load_metric
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

In [ ]:
nltk.download('punkt')

## Load the pubmed summarization dataset

In [ ]:
dataset = load_dataset("ccdv/pubmed-summarization")

In [ ]:
print(f"Features: {dataset['train'].column_names}")

In [ ]:
sample = dataset['train'][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample['article'][:500])
print(f'\nSummary (length: {len(sample["abstract"])}):')
print(sample["abstract"])

## Load the pretrained models

We could also load these models using `AutoModel` and `AutoTokenizer` to find the maximum input length. It would look something like:

```python
from transformers import AutoModel, AutoTokenizer

# Load the model and tokenizer
model_name = 'Falconsai/medical_summarization'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Get the maximum input size
max_input_size = model.config.max_position_embeddings
print(f"The maximum input size of the model is: {max_input_size}")
```

In [ ]:
summarizer_falconsai = pipeline("summarization", model="Falconsai/medical_summarization")

In [ ]:
summarizer_longt5 = pipeline("summarization", model="Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

## Sample text comparison

Here we'll take the first 3000 characters in order to have the same input for each of the models

In [ ]:
sample_text = sample['article'][:5000]
summaries = {}

In [ ]:
sample_text

## Evaluate pretrained models

As a baseline we will use the first three sentences as the summary

In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

#### Baseline

In [ ]:
summaries["baseline"] = three_sentence_summary(sample_text)

In [ ]:
summaries["baseline"]

#### Falconsai

In [ ]:
output = summarizer_falconsai(sample_text)

In [ ]:
output

In [ ]:
summaries["falconsai"] = "\n".join(sent_tokenize(output[0]["summary_text"]))

#### long t5

In [ ]:
output = summarizer_longt5(sample_text, max_length=200)

In [ ]:
output

In [ ]:
summaries["longt5"] = "\n".join(sent_tokenize(output[0]["summary_text"]))

## Comparing different summaries

In [ ]:
print("GROUND TRUTH")
print(sample['abstract'])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

## Evaluate quality of summaries

We will use the ROUGE metric to compare the quality of different summaries

In [ ]:
rouge_metric = load_metric("rouge")

In [ ]:
reference = sample['abstract']
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

## Evaluating the models on the validation dataset

To keep the evaluation time reasonable we will choose a subset of 200 examples from the test dataset to compare the models

In [ ]:
dataset

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1600))

#### Baseline

In [ ]:
def evaluate_summaries_baseline(dataset, metric,
                               column_text='article',
                               column_summary='abstract'):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries,
                     references=dataset[column_summary])
    score = metric.compute()
    return score

In [ ]:
score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

#### falconsai

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i + batch_size]

In [ ]:
def evaluate_summaries(dataset, metric, model, tokenizer,
                      batch_size=16, device=device,
                      column_text="article", column_summary="abstract"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for i, (article_batch, target_batch) in enumerate(zip(article_batches, target_batches)):
        if i % 10 == 0:
            print(f'Running batch {i + 1} of {len(article_batches)}')

        inputs = tokenizer(article_batch, max_length=2048, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=256)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_space=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
model_ckpt = "Falconsai/medical_summarization"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
score = evaluate_summaries(test_sampled, rouge_metric, model, tokenizer, batch_size=8)

In [ ]:
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["falconsai"])